# Import Python libraries

In [144]:
import xlwings as xw
import pandas as pd
import numpy as np
from poes.model.utils import param_poes
from scipy.stats import norm, lognorm, expon, triang, uniform

In [129]:
xb = xw.Book('poes/controller/control.xlsm')

In [130]:
sheet = xb.sheets['Resumen']

In [131]:
df_poes = sheet['A2'].options(pd.DataFrame, index=False, expand='table').value
df_poes

,Variables,Data values,Dist.,Loc,Scale,Sc,Lim. Min.,Lim. Max.
0,Area,340.00,None,None,None,None,None,None
1,Espesor,40.00,None,None,None,None,None,None
2,Porosida,0.16,None,None,None,None,None,None
3,Swi,0.51,None,None,None,None,None,None
4,Boi,2.60,None,None,None,None,None,None


# Exploratory data analysis

In [132]:
df_poes.columns

Index(['Variables', 'Data values', 'Dist.', 'Loc', 'Scale', 'Sc', 'Lim. Min. ',
       'Lim. Max.'],
      dtype='object')

In [133]:
df_poes.columns = df_poes.columns.str.strip()

In [134]:
df_poes.columns

Index(['Variables', 'Data values', 'Dist.', 'Loc', 'Scale', 'Sc', 'Lim. Min.',
       'Lim. Max.'],
      dtype='object')

In [135]:
df_poes['Dist.'] = ['Triangular', 'Triangular', 'Log Normal', 'Normal', 'Exponencial']
df_poes

,Variables,Data values,Dist.,Loc,Scale,Sc,Lim. Min.,Lim. Max.
0,Area,340.00,Triangular,None,None,None,None,None
1,Espesor,40.00,Triangular,None,None,None,None,None
2,Porosida,0.16,Log Normal,None,None,None,None,None
3,Swi,0.51,Normal,None,None,None,None,None
4,Boi,2.60,Exponencial,None,None,None,None,None


In [136]:
df_poes.shape

(5, 8)

In [137]:
df_poes['Loc'] = np.array([400, 0, 0, 0.4, 1])
df_poes['Scale'] = np.array([600, 90, 0.2, 0.2, 0.2])
df_poes['Sc'] = np.array([0.3, 0.3, 0.8, 0, 0.7])
df_poes['Lim. Min.'] = np.array([50, 0, 0, 0, 1])
df_poes['Lim. Max.'] = np.array([0, 180, 0.4, 1, 2])
df_poes

,Variables,Data values,Dist.,Loc,Scale,Sc,Lim. Min.,Lim. Max.
0,Area,340.00,Triangular,400.0,600.0,0.3,50,0.0
1,Espesor,40.00,Triangular,0.0,90.0,0.3,0,180.0
2,Porosida,0.16,Log Normal,0.0,0.2,0.8,0,0.4
3,Swi,0.51,Normal,0.4,0.2,0.0,0,1.0
4,Boi,2.60,Exponencial,1.0,0.2,0.7,1,2.0


# Use Loc method from pandas to extract specific values

In [138]:
df_poes.loc[2, "Data values"]

0.16

In [139]:
df_poes.loc[3, "Variables"]

'Swi'

# Call param_poes function

In [140]:
df_poes.columns

Index(['Variables', 'Data values', 'Dist.', 'Loc', 'Scale', 'Sc', 'Lim. Min.',
       'Lim. Max.'],
      dtype='object')

In [141]:
df_poes

,Variables,Data values,Dist.,Loc,Scale,Sc,Lim. Min.,Lim. Max.
0,Area,340.00,Triangular,400.0,600.0,0.3,50,0.0
1,Espesor,40.00,Triangular,0.0,90.0,0.3,0,180.0
2,Porosida,0.16,Log Normal,0.0,0.2,0.8,0,0.4
3,Swi,0.51,Normal,0.4,0.2,0.0,0,1.0
4,Boi,2.60,Exponencial,1.0,0.2,0.7,1,2.0


In [149]:
param_poes(df_poes, 3, 'Dist.', 'Loc', 'Scale', 100, 'Sc', 'Lim. Min.', 'Lim. Max.')

In [143]:
def param_stoiip(
    df,
    row,
    dist_col,
    loc_col,
    scale_col,
    iter,
    sc_col=None,
    lim_min_col=None,
    lim_max_col=None,
    seed=None,
):
    """
    This function has as a goal, return numpy arrays of any stoiip variable like area,
    thickness, porosity, oil saturation and Boi. Furthermore, this numpy array contains
    random variables from various continuous distributions.

    Parameters
    ----------
    df
        Pandas dataframe
    row
        row index
    dist_col
        distribution column
    loc_col
        loc argument column from scipy
    scale_col
        scale argument column from scipy
    sc_col
        c or s argument column from scipy
    lim_min_col
        Minimum limit column
    lim_max_col
        Maximin limit column
    iter
        Number of random variables
    seed
        Specifies the seed number to create the random number generator

    Returns
    -------
    numpy array of the any Stoiip parameter

    """

    if seed is None:
        rng = np.random.default_rng()
    else:
        rng = np.random.default_rng(seed)

    if df.loc[row, dist_col] == "Log-Normal" or df.loc[row, dist_col] == "Triangular":
        if df.loc[row, dist_col] == "Log-Normal":
            param = lognorm.rvs(
                s=df.loc[row, sc_col],
                loc=df.loc[row, loc_col],
                scale=df.loc[row, scale_col],
                size=iter,
                random_state=rng,
            )
            param = np.where(
                param < df.loc[row, lim_min_col], df.loc[row, lim_min_col], param
            )
            param = np.where(
                param > df.loc[row, lim_max_col], df.loc[row, lim_max_col], param
            )

        elif df.loc[row, dist_col] == "Triangular":
            param = triang.rvs(
                c=df.loc[row, sc_col],
                loc=df.loc[row, loc_col],
                scale=df.loc[row, scale_col],
                size=iter,
                random_state=rng,
            )

    elif df.loc[row, dist_col] == "Normal":
        param = norm.rvs(
            loc=df.loc[row, loc_col],
            scale=df.loc[row, scale_col],
            size=iter,
            random_state=rng,
        )
        param = np.where(
            param < df.loc[row, lim_min_col], df.loc[row, lim_min_col], param
        )
        param = np.where(
            param > df.loc[row, lim_max_col], df.loc[row, lim_max_col], param
        )

    elif df.loc[row, dist_col] == "Exponencial":
        param = expon.rvs(
            loc=df.loc[row, loc_col],
            scale=df.loc[row, scale_col],
            size=iter,
            random_state=rng,
        )
        param = np.where(
            param > df.loc[row, lim_max_col],
            df.loc[row, lim_max_col],
            param,
        )

    elif df.loc[row, dist_col] == "Rectangular":
        param = uniform.rvs(
            loc=df.loc[row, loc_col],
            scale=df.loc[row, scale_col],
            size=iter,
            random_state=rng,
        )

    return param

In [148]:
param_stoiip(df_poes, 3, 'Dist.', 'Loc', 'Scale', 100, 'Sc', 'Lim. Min.', 'Lim. Max.')

array([0.48275735, 0.36856971, 0.07697352, 0.23075571, 0.68653769,
       0.43420576, 0.65837029, 0.7053924 , 0.15927028, 0.44859897,
       0.14020174, 0.46517141, 0.38824839, 0.97375868, 0.229729  ,
       0.49817262, 0.48512111, 0.12249559, 0.20625855, 0.6232184 ,
       0.49495866, 0.48212433, 0.52764265, 0.29974031, 0.17359368,
       0.33144637, 0.10837005, 0.48828432, 0.5723086 , 0.78953157,
       0.        , 0.50556503, 0.38507287, 0.58813851, 0.69104246,
       0.18708673, 0.50101976, 0.67225353, 0.23443247, 0.1925057 ,
       0.43085725, 0.5926547 , 0.24064211, 0.24290324, 0.20965749,
       0.06053067, 0.65657121, 0.41550891, 0.45000914, 0.69931955,
       0.53007514, 0.36615253, 0.38307951, 0.62082304, 0.36138149,
       0.42543231, 0.27232759, 0.61655126, 0.26927861, 0.74349765,
       0.25647467, 0.20667166, 0.47056817, 0.14695612, 0.43760604,
       0.46416515, 0.36649164, 0.50033168, 0.29668216, 0.1341708 ,
       0.15411216, 0.40005854, 0.3808439 , 0.27276876, 0.51056